## Пример работы с CH

In [11]:
import pandas as pd

In [9]:
# блок, где генерируются данные для вставки в таблицу test_table
ids_arr = []
names_arr = []
amount_arr = []
for i in range(10000):
    ids_arr.append(i + 1)
    names_arr.append('Pskov')
    amount_arr.append((i + 1) * 34)
for i in range(10000, 20000):
    ids_arr.append(i + 1)
    names_arr.append('Murmansk')
    amount_arr.append((i + 1) * 34)

In [13]:
df_dict = {'id': ids_arr, 'name': names_arr, 'amount': amount_arr}
df = pd.DataFrame.from_dict(df_dict).set_index('id')
df.to_csv('example_data.csv', encoding='utf-8')

In [5]:
from clickhouse_driver.client import Client

In [7]:
# Подключение к серверу
client = Client('3.23.248.232', user='default', password='qwerty') # наш первый сервер
#client = Client('3.23.221.144', user='default', password='qwerty') # наш второй сервер
print(client.execute('SHOW DATABASES'))

[('_temporary_and_external_tables',), ('default',), ('system',), ('test_db',)]


### Структура тестовой базы test_db

В базе test_db на каждом сервере есть локальная таблица `test_table`. Обращение к ней: `test_db.test_table`.

Эти локальные таблицы имеют общую распределенную - `distr_test_table`. Обращение к ней: `test_db.distr_test_table`.

Поля таблицы `test_table`:
* id - Int64
* name - String, not null
* amount - Int64

Данные грузим в распределенную, потом они сами распределяются по локальным.

In [19]:
# Пример вставки данных в тестовую таблицу
client.execute('INSERT INTO test_db.distr_test_table VALUES', 
               (el for el in zip(ids_arr, names_arr, amount_arr)), 
               types_check=True)

20000

In [21]:
# Запрос к локальной таблице
client.execute('select count(*) from test_db.test_table')

[(24834,)]

In [22]:
# Запрос к распределенной таблице
client.execute('select count(*) from test_db.distr_test_table')

[(50000,)]

In [23]:
# Разрываем соединение с сервером
client.disconnect()